In [ ]:
%pip install -r requirements.txt --no-cache-dir

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *

from statsforecast import StatsForecast
from statsforecast.models import Naive, HistoricAverage, WindowAverage, SeasonalNaive, AutoARIMA

import warnings
warnings.filterwarnings("ignore")

***Exogenous Features:*** External variables that are not part of the time series itself.  
* Values of another series
* Holidays
* Any external factor

1. **Static Features:** Stays constant in time (product category).
2. **Historical Features:** Values are known in the past, but are unknown in the future (price of gas).
3. **Future Features:** Values are known both in the past and in the future (date of national holidays).

*Example:* Forecasting electricity demand; [ Outside temperature - Wind speed - Hour of the day ]

In [ ]:
df = pd.read_csv("daily_sales_french_bakery.csv", parse_dates=["ds"])
df = df.groupby("unique_id").filter(lambda x: len(x) >= 28)
df.head()

In [ ]:
unique_ids = ["BAGUETTE", "CROISSANT"]
small_df = df[df["unique_id"].isin(unique_ids)]
test = small_df.groupby("unique_id").tail(7)
train = small_df.drop(test.index).reset_index(drop=True)

In [ ]:
train.tail()

In [ ]:
horizon = 7

models = [
    AutoARIMA(season_length=7, alias="SARIMA")
]

sf = StatsForecast(models=models, freq="D")
sf.fit(df=train.drop(["unit_price"], axis=1))
arima_preds = sf.predict(h=horizon)

arima_preds

In [ ]:
results_df = test.merge(arima_preds, how="left", on=["unique_id", "ds"])
results_df

In [ ]:
plot_series(
    df=train,
    forecasts_df=results_df,
    ids=unique_ids,
    max_insample_length=28,
    palette="viridis",
    models=["SARIMA"]
)

In [ ]:
future_prices_df = test.drop(["y"], axis=1)
future_prices_df.head()

In [ ]:
models = [
    AutoARIMA(season_length=7, alias="SARIMA_price")
]

sf = StatsForecast(models=models, freq="D")
sf.fit(df=train)
arima_price_preds = sf.predict(h=horizon, X_df=future_prices_df)

results_df = results_df.merge(arima_price_preds, how="left", on=["unique_id", "ds"])

In [ ]:
plot_series(
    df=train,
    forecasts_df=results_df,
    ids=unique_ids,
    max_insample_length=28,
    palette="viridis",
    models=["SARIMA", "SARIMA_price"]
)

In [ ]:
models = [
    AutoARIMA(season_length=7, alias="SARIMA_price_crossval")
]

sf = StatsForecast(models=models, freq="D")

cv_exog_df = sf.cross_validation(
    h=horizon,
    df=small_df,
    n_windows=8,
    step_size=7,
    refit=True
)

cv_exog_df.head()

In [ ]:
cv_exog_eval = evaluate(
    cv_exog_df.drop(["cutoff"], axis=1),
    metrics=[mae]
)
cv_exog_eval = cv_exog_eval.drop(["unique_id"], axis=1).groupby("metric").mean().reset_index()
cv_exog_eval

***Creating Features from Timestamps***

In [ ]:
from functools import partial
from utilsforecast.feature_engineering import fourier, time_features, pipeline

features = [
    partial(fourier, season_length=7, k=2),
    partial(time_features, features=["day", "week", "month"])
]

small_exog_df, future_exogenous_df = pipeline(
    df=small_df,
    features=features,
    freq="D",
    h=horizon
)

In [ ]:
small_exog_df.head()

In [ ]:
future_exogenous_df.head()

In [ ]:
models = [
    AutoARIMA(season_length=7, alias="SARIMA_time_crossval")
]

sf = StatsForecast(models=models, freq="D")
cv_time_exog_df = sf.cross_validation(
    h=horizon,
    df=small_exog_df,
    n_windows=8,
    step_size=horizon,
    refit=True
)

cv_time_exog_eval = evaluate(
    cv_time_exog_df.drop(["cutoff"], axis=1),
    metrics=[mae]
)
cv_time_exog_eval = cv_time_exog_eval.drop(["unique_id"], axis=1).groupby("metric").mean().reset_index()
cv_time_exog_eval

In [ ]:
methods = ["SeasonalNaive", "ARIMA", "SARIMA", "SARIMA_price_crossval", "SARIMA_time_crossval"]
values = [21.117857, 21.17093, 19.281296, 19.210264, 19.532809]

sorted_data = sorted(zip(methods, values), key=lambda x: x[1], reverse=True)
methods_sorted, values_sorted = zip(*sorted_data)

plt.figure(figsize=(10, 6))
bars = plt.bar(methods_sorted, values_sorted)

for bar, value in zip(bars, values_sorted):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height()+0.05,
             f"{value:.3f}", ha="center", va="bottom", fontweight="bold")
    
plt.xlabel("Methods")
plt.ylabel("Mean Absolute Error (MAE)")
plt.tight_layout()
plt.show()